In [1]:
from helpers import *
from dataset import *
from model2_for48 import *
from czii_helper import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import random
import torchvision.transforms as T
import cv2
import math
import cc3d
import optuna
from torch.utils.data import Dataset, DataLoader
from monai.transforms import Compose, NormalizeIntensityd, ToTensorD
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import precision_score, recall_score, f1_score
from monai.transforms import RandFlipd,RandRotated,RandZoomd,RandAffined,ResizeD,SpatialPadd,RandRotate90d,EnsureChannelFirstd,Orientationd,RandAdjustContrastd,RandGaussianNoised

In [2]:
#DEBAG = False
DEBAG = True

In [3]:
MODE = 'local'
valid_dir = '../input/czii-cryo-et-object-identification/train'

cfg = dotdict(
    arch ='resnet34d',
    checkpoint=\
    '/kaggle/input/hengck-czii-cryo-et-weights-01/resnet34d-00010164-extern.pth',
   # '/kaggle/input/hengck-czii-cryo-et-weights-01/resnet34d-aug-noise-00003956.pth',
       #  '/kaggle/input/hengck-czii-cryo-et-weights-01/resnet34d-00002300.pth',
       # '/kaggle/input/hengck-czii-cryo-et-weights-01/00003531.pth',
    threshold={ 
        'apo-ferritin': 0.05,
        'beta-amylase': 0.05,
        'beta-galactosidase': 0.05,
        'ribosome': 0.05,
        'thyroglobulin': 0.05,
        'virus-like-particle': 0.05,
    }, 
)

In [4]:
LAMBDA = 0.0001
def objective(trial):
    # 各パラメータを最適化する
    mask_size2 = {
        'defolt': 1,
        'apo-ferritin': trial.suggest_float("apo-ferritin", 0.5, 1.0) * mask_size["apo-ferritin"],
        'beta-amylase': 0.01,
        'beta-galactosidase': trial.suggest_float("beta-galactosidase", 0.5, 1) * mask_size["beta-galactosidase"],
        'ribosome': trial.suggest_float("ribosome", 0.5, 1.0) * mask_size["ribosome"],
        'thyroglobulin': trial.suggest_float("thyroglobulin", 0.5, 1.0) * mask_size["thyroglobulin"],
        'virus-like-particle': trial.suggest_float("virus-like-particle", 0.5, 1.0) * mask_size["virus-like-particle"],
    }

    # IDリスト
    id_list = ['TS_99_9', "TS_6_4", "TS_6_6", "TS_69_2", "TS_73_6", "TS_86_3","TS_5_4"]
    
    # 評価スコアの計算
    lb_scores = []
    for ids in id_list:
        _, lb_score = mask_check(ids, mask_size2, cfg)
        lb_scores.append(lb_score)

    mean_lb_score = np.mean(lb_scores)
    total_mask_size = sum(mask_size2.values())
    
    return mean_lb_score + LAMBDA * total_mask_size

# Optunaの最適化
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# 最適なパラメータを表示
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)

[I 2025-01-30 03:52:46,605] A new study created in memory with name: no-name-f5be6eac-7768-4b9a-a077-d5084072bf1d


['TS_99_9']
 TS_99_9 virus-like-particle
['TS_6_4']
 TS_6_4 virus-like-particle
['TS_6_6']
 TS_6_6 virus-like-particle
['TS_69_2']
 TS_69_2 virus-like-particle
['TS_73_6']
 TS_73_6 virus-like-particle
['TS_86_3']
 TS_86_3 virus-like-particle
['TS_5_4']
 TS_5_4 virus-like-particle

[I 2025-01-30 03:53:34,826] Trial 0 finished with value: 0.9548146887341838 and parameters: {'apo-ferritin': 0.5540628338304592, 'beta-galactosidase': 0.8561602404541752, 'ribosome': 0.9009110195270496, 'thyroglobulin': 0.539415289432817, 'virus-like-particle': 0.9536148566067072}. Best is trial 0 with value: 0.9548146887341838.



['TS_99_9']
 TS_99_9 virus-like-particle
['TS_6_4']
 TS_6_4 virus-like-particle
['TS_6_6']
 TS_6_6 virus-like-particle
['TS_69_2']
 TS_69_2 virus-like-particle
['TS_73_6']
 TS_73_6 virus-like-particle
['TS_86_3']
 TS_86_3 virus-like-particle
['TS_5_4']
 TS_5_4 virus-like-particle

[I 2025-01-30 03:54:20,624] Trial 1 finished with value: 0.9901602208223712 and parameters: {'apo-ferritin': 0.6437435569400045, 'beta-galactosidase': 0.5409054497908696, 'ribosome': 0.6882053227363547, 'thyroglobulin': 0.755307191882315, 'virus-like-particle': 0.7476425471857988}. Best is trial 1 with value: 0.9901602208223712.


...


['TS_99_9']
 TS_99_9 virus-like-particle
['TS_6_4']
 TS_6_4 virus-like-particle
['TS_6_6']
 TS_6_6 virus-like-particle
['TS_69_2']
 TS_69_2 virus-like-particle
['TS_73_6']
 TS_73_6 virus-like-particle
['TS_86_3']
 TS_86_3 virus-like-particle
['TS_5_4']
 TS_5_4 virus-like-particle

[I 2025-01-30 04:30:00,900] Trial 48 finished with value: 0.999273218899391 and parameters: {'apo-ferritin': 0.5007696967945715, 'beta-galactosidase': 0.842713283404142, 'ribosome': 0.6695325096638413, 'thyroglobulin': 0.5612182896421284, 'virus-like-particle': 0.996387384261614}. Best is trial 47 with value: 1.0023438893745318.



['TS_99_9']
 TS_99_9 virus-like-particle
['TS_6_4']
 TS_6_4 virus-like-particle
['TS_6_6']
 TS_6_6 virus-like-particle
['TS_69_2']
 TS_69_2 virus-like-particle
['TS_73_6']
 TS_73_6 virus-like-particle
['TS_86_3']
 TS_86_3 virus-like-particle
['TS_5_4']
 TS_5_4 virus-like-particle

[I 2025-01-30 04:30:48,113] Trial 49 finished with value: 0.9418580222769791 and parameters: {'apo-ferritin': 0.5229266816652135, 'beta-galactosidase': 0.9232330554049846, 'ribosome': 0.9368925800245198, 'thyroglobulin': 0.5248672157677008, 'virus-like-particle': 0.8147497989285986}. Best is trial 47 with value: 1.0023438893745318.



Best parameters: {'apo-ferritin': 0.5002733727905936, 'beta-galactosidase': 0.8560637150342462, 'ribosome': 0.6675785100335182, 'thyroglobulin': 0.5012172476510953, 'virus-like-particle': 0.7247879252485406}
Best score: 1.0023438893745318


In [7]:
mask_size2

{'defolt': 1,
 'apo-ferritin': 3.0016402367435617,
 'beta-amylase': 0.01,
 'beta-galactosidase': 7.704573435308215,
 'ribosome': 10.013677650502773,
 'thyroglobulin': 6.515824219464239,
 'virus-like-particle': 9.784636990855299}

In [8]:
best_params

{'apo-ferritin': 0.5002733727905936,
 'beta-galactosidase': 0.8560637150342462,
 'ribosome': 0.6675785100335182,
 'thyroglobulin': 0.5012172476510953,
 'virus-like-particle': 0.7247879252485406}

In [6]:
# 最適なパラメータを取得
best_params = study.best_params

# デバッグ用 mask_size2 の作成
mask_size2 = {
    'defolt': 1,
    'apo-ferritin': best_params["apo-ferritin"] * mask_size["apo-ferritin"],
    'beta-amylase': 0.01,  # 固定値
    'beta-galactosidase': best_params["beta-galactosidase"] * mask_size["beta-galactosidase"],
    'ribosome': best_params["ribosome"] * mask_size["ribosome"],
    'thyroglobulin': best_params["thyroglobulin"] * mask_size["thyroglobulin"],
    'virus-like-particle': best_params["virus-like-particle"] * mask_size["virus-like-particle"],
}

id_list = ['TS_99_9', "TS_6_4", "TS_6_6", "TS_69_2", "TS_73_6", "TS_86_3","TS_5_4"]
for ids in id_list:
    lb, lb_score = mask_check(ids, mask_size2, cfg)
    print(lb)

['TS_99_9']
 TS_99_9 virus-like-particle
         particle_type   P   T  hit  miss  fp  precision  recall  f-beta4  \
0         apo-ferritin  36  36   36     0   0        1.0     1.0      1.0   
1         beta-amylase  21  21   21     0   0        1.0     1.0      1.0   
2   beta-galactosidase  24  24   24     0   0        1.0     1.0      1.0   
3             ribosome  65  65   65     0   0        1.0     1.0      1.0   
4        thyroglobulin  49  49   49     0   0        1.0     1.0      1.0   
5  virus-like-particle  13  13   13     0   0        1.0     1.0      1.0   

   weight  
0       1  
1       0  
2       2  
3       1  
4       2  
5       1  
['TS_6_4']
 TS_6_4 virus-like-particle
         particle_type   P   T  hit  miss  fp  precision    recall   f-beta4  \
0         apo-ferritin  58  58   58     0   0   1.000000  1.000000  1.000000   
1         beta-amylase   8   9    8     1   0   1.000000  0.888889  0.894737   
2   beta-galactosidase  12  12   12     0   0   1.000000

In [14]:
mask_size2

{'apo-ferritin': 0.3975137043063831,
 'beta-galactosidase': 0.8351340745406122,
 'ribosome': 0.9397823772094265,
 'thyroglobulin': 0.5643048741101144,
 'virus-like-particle': 0.8332201163209894,
 'beta-amylase': 0.01}